<a href="https://colab.research.google.com/github/anandababugudipudi/TensorFlow-ML/blob/main/3_Core_Learning_Algos_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Classificatin Problem on IRIS Flower Dataset**


In [ ]:
# Importing the necessary modules
%tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

**IRIS Flower Dataset**


In [ ]:
# Features and Labels
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# Load dataset.
XTrain = pd.read_csv("https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv", names = CSV_COLUMN_NAMES, header = 0)
XTest = pd.read_csv("https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv", names = CSV_COLUMN_NAMES, header = 0)
print(XTrain)

In [ ]:
XTrain.head()

In [ ]:
# Loading the Labels
yTrain = XTrain.pop('Species')
yTest = XTest.pop('Species')
print(yTrain.shape)
print(yTest.shape)

In [ ]:
# Creating an input function
def input_fn(features, labels, training = True, batch_size = 256):
  # Covnert the inputs into a Dataset
  ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
  if training:
    ds = ds.shuffle(1000).repeat()
  return ds.batch(batch_size)

# Creating feature columns
feature_columns = []
for key in XTrain.keys():
  feature_columns.append(tf.feature_column.numeric_column(key = key))
print(feature_columns)

In [ ]:
# Building the classifier model
# Here we are building a Deep Neural Network Classifier with 30 and 10 hidden nodes eachother
# We will be using n_classes = 3 as we have to classify among 3 classes
model = tf.estimator.DNNClassifier(
    feature_columns = feature_columns,
    hidden_units = [30, 10], 
    n_classes = 3)

In [ ]:
# Training the model
model.train(
    input_fn = lambda : input_fn(XTrain, yTrain, training = True),
    steps = 7000)

In [ ]:
# Evaluating the model
eval_results = model.evaluate(
    input_fn = lambda : input_fn(XTest, yTest, training = False))
clear_output()
print(f"The accuracy is {round(eval_results['accuracy']*100,2)}")

In [ ]:
# Predicting the model with user inputs
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = model.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))
